# News Parsing Version 1

## Рекомендуемая структура проекта

In [ ]:
news_parser/
│
├── .env                    # Секретные переменные (в .gitignore)
├── .gitignore              # Игнорируемые файлы
├── requirements.txt        # Зависимости
├── pyproject.toml          # Альтернатива requirements.txt (опционально)
│
├── config/                 
│   ├── __init__.py         # Пустой файл для импортов
│   ├── settings.py         # Общие настройки (несекретные)
│   ├── telegram.py         # Настройки бота
│   └── sites/              # Конфиги для разных сайтов
│       ├── vc_ru.json      
│       └── rbc.json        
│
├── data/
│   ├── raw/                # Сырые данные (кеш HTML)
│   │   ├── vc_ru/         
│   │   └── rbc/           
│   │
│   ├── interim/            # Промежуточные данные
│   │   └── dynamic_page.html  
│   │
│   ├── processed/          # Обработанные данные
│   │   ├── articles.csv    
│   │   └── articles.json   
│   │
│   ├── encrypted/          # Зашифрованные данные (если нужно)
│   ├── backups/            # Автоматические бэкапы
│   └── state/              # Состояния для возобновления
│       └── last_failure.pkl  
│
├── logs/                   # Логи работы
│   ├── parsing_20240515.log  
│   └── errors.log          
│
├── parsers/                # Модули парсинга
│   ├── __init__.py         
│   ├── base_parser.py      # Базовый класс
│   ├── vc_ru.py           # Парсер для VC.ru
│   ├── rbc.py             # Парсер для RBC
│   └── utils/             
│       ├── anti_block.py   # Обход блокировок
│       └── helpers.py      # Вспомогательные функции
│
├── storage/                # Работа с хранилищами
│   ├── __init__.py         
│   ├── local.py           # CSV/JSON
│   ├── google_sheets.py   
│   └── database.py        # Для будущей БД
│
├── notifications/          # Уведомления
│   ├── __init__.py         
│   ├── telegram.py        
│   └── email.py           # Резервный канал
│
├── main.py                 # Точка входа
└── run.bat                 # Скрипт запуска (Windows)

### Ключевые особенности

#### 1. Безопасность данных:
- Все секреты (токены, API-ключи) хранятся в .env (добавлен в .gitignore)
- Пример .env:

In [ ]:
# ini
TELEGRAM_TOKEN=your_telegram_token
GSHEETS_CREDENTIALS=credentials.json
DB_PASSWORD=secret

#### 2. Резервное копирование:
- Автоматические бэкапы в data/backups/ с датой в имени файла:

In [ ]:
# python
shutil.copy2("data/processed/articles.csv", f"data/backups/articles_{datetime.now().date()}.csv")

#### 3. Шифрование (опционально):
- Для чувствительных данных используйте data/encrypted/ с библиотекой cryptography.

#### 4. Логирование:
- Настройка в main.py:

In [ ]:
# python
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("logs/parsing.log"),
        logging.StreamHandler()
    ]
)

#### 5. Пример .gitignore

In [ ]:
# gitignore
# Секреты
.env
credentials.json
data/encrypted/*

# Виртуальное окружение
venv/
.env/

# Кеши и временные файлы
__pycache__/
*.py[cod]
data/raw/
data/interim/
data/state/

# IDE
.idea/
.vscode/

#### 6. Как начать использовать
- 1. Создайте структуру папок:

In [ ]:
# bash
mkdir -p news_parser/{config/sites,data/{raw,interim,processed,backups,state},logs,parsers/utils,storage,notifications}

In [ ]:
# - 2. Установите зависимости:
# bash
pip install -r requirements.txt

In [ ]:
# - 3. Заполните .env по образцу .env.example (создайте его для примера).

#### 7. Следующие шаги
Когда будете готовы продолжить, мы можем:

- Реализовать систему плавного возобновления после сбоев.
- Добавить мониторинг изменений на сайтах (алерт при изменении структуры).
- Настроить автоматические бэкапы в облако (S3, Yandex Disk).

## 0. Виртуальное окружение (обязательно!)

### Перед созданием файлов нужно изолировать зависимости проекта с помощью venv или conda:

In [ ]:
# bash
python -m venv venv  # Создаем виртуальное окружение
source venv/bin/activate  # Активируем (Linux/Mac)
venv\Scripts\activate     # Активируем (Windows)

### 2. Файл requirements.txt
Все библиотеки из вашего кода нужно сохранить в файле requirements.txt:

In [ ]:
# text
# requirements.txt
beautifulsoup4==4.12.0
requests==2.31.0
selenium==4.15.0
webdriver-manager==4.0.0
pandas==2.1.0
gspread==5.11.0
telebot==0.0.5
python-dotenv==1.0.0  # Для загрузки переменных окружения (например, токенов)

In [ ]:
# Команда для установки:
# bash
pip install -r requirements.txt

### 3. Альтернатива: pyproject.toml (для современных проектов)
Если проект сложный, можно использовать poetry или pipenv. Пример для pyproject.toml:

In [ ]:
# toml
[project]
dependencies = [
    "beautifulsoup4>=4.12.0",
    "selenium>=4.15.0",
    "gspread>=5.11.0",
]

### 4. Отдельный файл для констант/настроек
Библиотеки можно разделить на логические группы в файле config.py или constants.py:

In [ ]:
# python
# config.py
# Основные
import os
import json
from datetime import datetime

# Парсинг
from bs4 import BeautifulSoup
import requests

# Динамические сайты
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Google Sheets
import gspread

### 5. Где хранить токены/ключи?
Никогда не кладите их прямо в код! Используйте:
Файл .env (через библиотеку python-dotenv):

In [ ]:
# text
# .env
TELEGRAM_BOT_TOKEN=your_token_here
GSHEET_API_KEY=your_key_here

#### Или настройки ОС (через os.environ).

### 6. Рекомендации:
#### 1. gitignore:
Добавьте исключения для виртуального окружения и секретных файлов:

In [ ]:
# text
venv/
.env
*.bat
__pycache__/

#### 2. Автоматизация:
Для .bat-файла (Windows) можно добавить команды активации окружения и запуска:

In [ ]:
# bat
@echo off
call venv\Scripts\activate
python main.py
pause

In [ ]:
### 

## 1. Загрузка библиотек
Все библиотеки для работы проекта в одном месте

In [ ]:
# Загрузка библиотек
# Для вспомогательных функций и основного парсера
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.parse import urljoin

# Для экспорта/импорта временных данных
import json
import csv

# Для работы с динамическими сайтами
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
import random

# Обновление CSV/JSON с дедупликацией
import pandas as pd

# Google Sheets
from gspread import Client, Spreadsheet, Worksheet, service_account, exceptions
from typing import Optional, List, Dict

# Библиотеки для ТелеБОТа
import telebot
# для указание типов
from telebot import types
# токен лежит в файле config.py
import config

## 2. DYNAMIC_PARSE()
Динамически загружает контент с имитацией поведения человека.

#### Улучшения:
1. Гибкость настроек:
Вынесем параметры Selenium в конфиг (например, user_agent, timeout).

2. Без промежуточного HTML:
Убираем сохранение в файл (как вы и хотели).

3. Логирование:
Заменим print() на модуль logging.

Обновлённый код:

### 1. Модификация dynamic_parse()

In [ ]:
# 
from typing import Optional
import logging
from selenium.webdriver.remote.webdriver import WebDriver

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def dynamic_parse(
    url: str = "https://vc.ru/new",
    max_scrolls: int = 10,
    driver: Optional[WebDriver] = None
) -> Optional[str]:
    """
    Динамически загружает контент с имитацией поведения человека.
    
    Args:
        url: URL для парсинга.
        max_scrolls: Максимальное число прокруток.
        driver: Опционально, существующий экземпляр WebDriver.
        
    Returns:
        HTML-контент или None при ошибке.
    """
    local_driver = None
    try:
        if not driver:
            service = Service(ChromeDriverManager().install())
            options = webdriver.ChromeOptions()
            options.add_argument("--headless")  # Для продакшена без GUI
            options.add_argument("--disable-blink-features=AutomationControlled")
            options.add_argument(f"user-agent={get_random_user_agent()}")
            local_driver = webdriver.Chrome(service=service, options=options)
            driver = local_driver

        driver.get(url)
        
        for count in range(1, max_scrolls + 1):
            human_like_interaction(driver)
            logger.info(f"Прокрутка #{count} завершена")
            
            # Обработка кнопки "Показать ещё" (если есть)
            load_more_content(driver)

        logger.info("✅ Динамический контент загружен")
        return driver.page_source

    except Exception as e:
        logger.error(f"Ошибка в dynamic_parse: {type(e).__name__} - {e}", exc_info=True)
        return None
    finally:
        if local_driver:  # Закрываем только если создавали здесь
            local_driver.quit()

def get_random_user_agent() -> str:
    """Генерирует случайный User-Agent."""
    chrome_versions = range(90, 116)
    return f"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.choice(chrome_versions)}.0.0.0 Safari/537.36"

def load_more_content(driver: WebDriver) -> None:
    """Пытается найти и нажать кнопку 'Показать ещё'."""
    try:
        more_btn = driver.find_element(By.CSS_SELECTOR, ".button-more")
        if more_btn.is_displayed():
            more_btn.click()
            time.sleep(random.uniform(2, 4))
    except Exception:
        pass

### 2. Модификация human_like_interaction()

#### Улучшения:
1. Разделение логики:
Вынесем плавную прокрутку и hover в отдельные функции.

2. Конфигурируемость:
Параметры (паузы, вероятность hover) можно вынести в конфиг.

Обновлённый код:

In [ ]:
# 
def human_like_interaction(driver: WebDriver) -> None:
    """Имитирует действия человека."""
    smooth_scroll(driver)
    random_hover(driver)

def smooth_scroll(driver: WebDriver, scroll_amount: Optional[int] = None) -> None:
    """Плавная прокрутка страницы."""
    scroll_pause = random.uniform(3, 5)
    scroll_amount = scroll_amount or random.randint(800, 1200)
    driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
    time.sleep(scroll_pause)

def random_hover(driver: WebDriver, probability: float = 0.3) -> None:
    """Случайный hover на элементы (по умолчанию 30% вероятность)."""
    if random.random() > probability:
        elements = driver.find_elements(By.CSS_SELECTOR, ".content-title, .author__name, .content-header__item")
        if elements:
            element = random.choice(elements)
            ActionChains(driver).move_to_element(element).pause(1).perform()
            time.sleep(random.uniform(1, 2))

### Интеграция в проект

In [ ]:
# Пример использования:
from parsers.dynamic import dynamic_parse

html = dynamic_parse(max_scrolls=5)
if html:
    # Парсим "с лёту" без сохранения в файл
    soup = BeautifulSoup(html, 'html.parser')
    # ... обработка данных ...

In [ ]:
# 


### 4. Что дальше?
1. Тестирование:
Проверить работу с разными сайтами.
Добавить обработку капчи (если появится).

2. Оптимизация:
Кеширование драйвера (не создавать его каждый раз).

3. Безопасность:
Проверка url на валидность перед запросом.

In [ ]:
# bash


## 3. PARSE_ARTICLES
Запуск основной функции парсинга

### 1. ООП-подход: Создание парсеров для каждого сайта
Идея:
Создаем базовый класс BaseParser с общими методами, а для каждого сайта — наследника с конкретной реализацией.

#### 1.1. Базовая структура

In [ ]:
# 
from abc import ABC, abstractmethod
from typing import Dict, List, Optional
from bs4 import BeautifulSoup

class BaseParser(ABC):
    def __init__(self, host: str):
        self.host = host

    @abstractmethod
    def parse_articles(self, html: str) -> List[Dict]:
        """Основной метод парсинга. Должен быть реализован для каждого сайта."""
        pass

    # Общие вспомогательные методы (можно переопределять)
    def get_text_or_none(self, parent, selector: str) -> Optional[str]:
        element = parent.select_one(selector)
        return element.get_text(strip=True) if element else None

    def get_link(self, parent, selector: str) -> Optional[str]:
        element = parent.select_one(selector)
        return urljoin(self.host, element['href']) if element and 'href' in element.attrs else None

#### 1.2. Пример для vc.ru

In [ ]:
# 
class VcRuParser(BaseParser):
    def parse_articles(self, html: str) -> List[Dict]:
        soup = BeautifulSoup(html, 'html.parser')
        articles = soup.find('div', class_='content-list')
        if not articles:
            raise ValueError("Секция с новостями не найдена")

        results = []
        for article in articles.find_all('div', class_='content--short'):
            data = {
                'title': self.get_text_or_none(article, 'div.content-title'),
                'author': self.get_text_or_none(article, 'a.author__name'),
                'link': self.get_link(article, 'a.content__link'),
                'category': self._extract_category(article),
            }
            results.append(data)
        return results

    def _extract_category(self, article) -> Optional[str]:
        """Уникальный метод для vc.ru."""
        url = self.get_link(article, 'a.content__link')
        if not url:
            return None
        parts = url.replace(self.host, '').strip('/').split('/')
        return parts[0] if parts else None

#### 1.3. Использование

In [ ]:
# 
parser = VcRuParser(host="https://vc.ru")
html = dynamic_parse(url="https://vc.ru/new")
articles = parser.parse_articles(html)

### 2. Конфигурационный подход (без ООП)
Идея:
Храним CSS-селекторы для каждого сайта в конфиге (JSON/YAML), а парсер использует их динамически.

#### 2.1. Конфиг (config/sites/vc.ru.json):

In [ ]:
# json
{
  "host": "https://vc.ru",
  "selectors": {
    "articles_container": "div.content-list",
    "article": "div.content--short",
    "fields": {
      "title": "div.content-title",
      "author": "a.author__name",
      "link": "a.content__link"
    }
  },
  "custom_functions": {
    "category": "extract_vc_ru_category"
  }
}

#### 2.2. Универсальный парсер

In [ ]:
# 
def parse_with_config(html: str, config: Dict) -> List[Dict]:
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.select_one(config['selectors']['articles_container'])
    results = []

    for article in articles.select(config['selectors']['article']):
        data = {}
        for field, selector in config['selectors']['fields'].items():
            data[field] = get_text_or_none(article, selector)
        
        # Кастомная логика (например, категория)
        if 'custom_functions' in config:
            for field, func_name in config['custom_functions'].items():
                data[field] = globals()[func_name](article)
        results.append(data)
    return results

### 3. Гибридный подход (ООП + Конфиги)
Идея:
Комбинируем оба метода. Базовый класс парсера загружает конфиг, но позволяет переопределять сложную логику.

In [ ]:
# 
class ConfigurableParser(BaseParser):
    def __init__(self, config_path: str):
        with open(config_path) as f:
            self.config = json.load(f)
        super().__init__(self.config['host'])

    def parse_articles(self, html: str) -> List[Dict]:
        # Базовая логика из конфига
        data = parse_with_config(html, self.config)
        # Дополнительная обработка
        return self.post_process(data)

    def post_process(self, data: List[Dict]) -> List[Dict]:
        """Можно переопределить в дочерних классах."""
        return data

### 4. Обработка капчи и защит
Для обхода защит:
##### 1. Rotate User-Agents и прокси:
Используйте библиотеку fake-useragent и сервисы типа ScraperAPI.

##### 2. Задержки между запросами:

In [ ]:
# Задержки между запросами:
time.sleep(random.uniform(1, 3))

##### 3. Headless-браузер:
В Selenium используйте --headless, но добавляйте человекообразные действия (как у вас в human_like_interaction).

### Итог
- Для 1-3 сайтов — ООП-подход (классы-парсеры).
- Для 5+ сайтов — конфиги + возможность кастомной логики.
- Сложные сайты (капча, JS) — Selenium + обходные методы.

## 4. UPDATE_DATA
Обновление CSV/JSON с дедупликацией (с доработкой для работы с форматами времени)

### 1. Улучшенная версия update_data()

In [ ]:
# 
import pandas as pd
from pathlib import Path
from typing import List, Dict, Optional
import logging
from datetime import datetime

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def update_data(
    new_data: List[Dict],
    csv_path: str = "data/output/articles.csv",
    json_path: str = "data/output/articles.json",
    date_format: str = "%d.%m.%Y. %H:%M:%S",
    dedup_key: str = "title"
) -> Optional[pd.DataFrame]:
    """
    Обновляет данные с дедупликацией, сохраняя первую дату появления записи.
    
    Args:
        new_data: Список новых данных в виде словарей.
        csv_path: Путь к CSV-файлу.
        json_path: Путь к JSON-файлу.
        date_format: Формат даты для парсинга.
        dedup_key: Поле для дедупликации (по умолчанию 'title').
        
    Returns:
        DataFrame с обновлёнными данными или None при ошибке.
    """
    try:
        # Создаём директории, если их нет
        Path(csv_path).parent.mkdir(parents=True, exist_ok=True)
        
        # Загрузка старых данных
        old_df = load_existing_data(csv_path, date_format)
        
        # Обработка новых данных
        new_df = pd.DataFrame(new_data)
        new_df["parse_datetime"] = pd.to_datetime(
            new_df["parse_datetime"], 
            format=date_format,
            dayfirst=True
        )
        
        # Объединение и дедупликация
        combined_df = deduplicate_data(old_df, new_df, dedup_key)
        combined_df["parse_datetime"] = combined_df["parse_datetime"].dt.strftime(date_format)
        
        # Сохранение
        save_data(combined_df, csv_path, json_path)
        logger.info(f"Данные обновлены. Уникальных записей: {len(combined_df)}")
        return combined_df
        
    except Exception as e:
        logger.error(f"Ошибка в update_data: {e}", exc_info=True)
        return None

def load_existing_data(path: str, date_format: str) -> pd.DataFrame:
    """Загружает существующие данные из CSV."""
    try:
        df = pd.read_csv(path)
        df["parse_datetime"] = pd.to_datetime(
            df["parse_datetime"], 
            format=date_format,
            dayfirst=True
        )
        return df
    except FileNotFoundError:
        return pd.DataFrame()

def deduplicate_data(
    old_df: pd.DataFrame, 
    new_df: pd.DataFrame, 
    key: str
) -> pd.DataFrame:
    """Объединяет и удаляет дубликаты."""
    combined_df = pd.concat([old_df, new_df], ignore_index=True)
    combined_df["first_seen"] = combined_df.groupby(key)["parse_datetime"].transform("min")
    return (
        combined_df
        .sort_values("parse_datetime", ascending=False)
        .drop_duplicates(key, keep="first")
        .drop(columns="first_seen")
    )

def save_data(
    df: pd.DataFrame, 
    csv_path: str, 
    json_path: str
) -> None:
    """Сохраняет данные в CSV и JSON."""
    df.to_csv(csv_path, index=False, encoding='utf-8')
    df.to_json(json_path, orient="records", force_ascii=False, indent=2)

### 2. Подготовка к интеграции с БД

#### 2.1. Адаптер для БД
Добавим абстрактный класс для работы с разными хранилищами:

In [ ]:
# 
from abc import ABC, abstractmethod

class DataStorage(ABC):
    @abstractmethod
    def save(self, data: pd.DataFrame) -> bool:
        pass
    
    @abstractmethod
    def load(self) -> pd.DataFrame:
        pass

# Пример реализации для PostgreSQL
class PostgresStorage(DataStorage):
    def __init__(self, connection_string: str):
        self.conn = psycopg2.connect(connection_string)
        
    def save(self, data: pd.DataFrame) -> bool:
        try:
            data.to_sql("articles", self.conn, if_exists="replace", index=False)
            return True
        except Exception as e:
            logger.error(f"Ошибка сохранения в PostgreSQL: {e}")
            return False

# Пример для Git (версионирование данных)
class GitDataStorage:
    def __init__(self, repo_path: str):
        self.repo = git.Repo(repo_path)
        
    def commit_data(self, message: str):
        self.repo.git.add("data/")
        self.repo.index.commit(message)

### 3. Защита данных

#### 3.1. Шифрование чувствительных полей

In [ ]:
# 
from cryptography.fernet import Fernet

class DataEncryptor:
    def __init__(self, key: str):
        self.cipher = Fernet(key.encode())
        
    def encrypt_field(self, value: str) -> str:
        return self.cipher.encrypt(value.encode()).decode()
        
    def decrypt_field(self, value: str) -> str:
        return self.cipher.decrypt(value.encode()).decode()

In [ ]:
# 


#### 3.2. Пример использования

In [ ]:
# 
encryptor = DataEncryptor(os.getenv("ENCRYPTION_KEY"))
df["author"] = df["author"].apply(encryptor.encrypt_field)

### 4. Архитектура хранения

#### 4.1. Варианты размещения данных
Способ              Плюсы	                      Минусы
Локальные CSV/JSON	Простота, не требует сервера  Нет версионности, сложный поиск
PostgreSQL	        Поддержка сложных запросов	  Требует сервер
Git-репозиторий	    История изменений, бэкапы	  Медленная работа с большими файлами
Google Sheets	    Доступ из любого места	      Ограничения API

### 5. Интеграция с планировщиком

In [ ]:
# Пример .bat-файла для автоматического обновления:
# bat
@echo off
call venv\Scripts\activate
python -c "from main import update_all; update_all()"
git add data/processed/
git commit -m "Auto-update: %date% %time%"
git push

### 6. Дальнейшие шаги
1. Выбор хранилища:
Для старта хватит CSV + Git.
При росте данных — PostgreSQL.

2. Резервное копирование:
Автоматические бэкапы в облако (S3, Dropbox).

3. Мониторинг:
Логирование всех операций в logs/app.log.

## 5. Функция EXPORT_TO_GOOGLE_SHEETS
Код для "умного" обновления Google Sheets

### 1. Улучшенная версия функции

In [ ]:
# 
from typing import List, Dict, Optional, Union
from gspread import Worksheet
from google.oauth2.service_account import Credentials
import logging
from datetime import datetime

def export_to_google_sheets(
    data_source: Union[str, List[Dict]],  # Принимает JSON-файл или готовый список словарей
    spreadsheet_id: str,
    sheet_name: str = "News",
    credentials_path: str = "credentials.json",
    dedup_key: str = "title",  # Ключ для дедупликации
    batch_size: int = 1000,    # Размер пачки для обновления (избегаем лимитов API)
    time_format: str = "%d.%m.%Y %H:%M:%S"
) -> Optional[int]:
    """
    Экспортирует данные в Google Sheets с дедупликацией и поддержкой многолистовой структуры.
    
    Args:
        data_source: Путь к JSON-файлу или список словарей с данными.
        spreadsheet_id: ID Google Sheets.
        sheet_name: Название листа (или 'auto' для автоматического выбора).
        credentials_path: Путь к файлу учётных данных.
        dedup_key: Поле для дедупликации.
        batch_size: Максимальное количество строк для одного запроса API.
        
    Returns:
        Количество уникальных записей или None при ошибке.
    """
    try:
        # --- Инициализация ---
        creds = Credentials.from_service_account_file(credentials_path)
        gc = gspread.authorize(creds)
        spreadsheet = gc.open_by_key(spreadsheet_id)
        
        # --- Загрузка данных ---
        if isinstance(data_source, str):
            with open(data_source, 'r', encoding='utf-8') as f:
                new_data = json.load(f)
        else:
            new_data = data_source
            
        if not new_data:
            logging.warning("Нет данных для экспорта.")
            return 0

        # --- Автовыбор листа для мультисайтового парсинга ---
        if sheet_name == "auto":
            sheet_name = infer_sheet_name(new_data[0])  # Пример: берём категорию из данных
            
        worksheet = get_or_create_worksheet(spreadsheet, sheet_name)
        
        # --- Дедупликация ---
        old_data = worksheet.get_all_records()
        unique_data = merge_and_deduplicate(old_data, new_data, dedup_key)
        
        # --- Форматирование дат ---
        format_dates(unique_data, time_format)
        
        # --- Пакетная запись ---
        update_worksheet(worksheet, unique_data, batch_size)
        
        logging.info(f"Обновлено {len(unique_data)} записей в листе '{sheet_name}'")
        return len(unique_data)
        
    except Exception as e:
        logging.error(f"Ошибка экспорта: {e}", exc_info=True)
        return None

### 2. Ключевые улучшения

#### 2.1. Поддержка разных источников данных

Теперь функция принимает:
- Путь к JSON (data/output/articles.json)
- Готовый список словарей (результат parse_articles())

In [ ]:
# 
# Вариант 1: Из файла
export_to_google_sheets("data/output/articles.json", "123abc")

# Вариант 2: Напрямую из памяти
articles = parse_articles(html)
export_to_google_sheets(articles, "123abc")

#### 2.2. Автоматический выбор листа
Для парсинга разных сайтов/категорий:

In [ ]:
# 
def infer_sheet_name(item: Dict) -> str:
    """Определяет имя листа на основе данных."""
    return item.get("category", "General").replace("/", "_")[:30]  # Ограничение Google

In [ ]:
# Пример использования:
# Создаст листы "VC_RU", "RBC" и т.д.
export_to_google_sheets(data, sheet_name="auto")

In [ ]:
#### 2.3. Пакетная запись (batch update)
Избегаем ограничений Google Sheets API:

In [ ]:
# 
def update_worksheet(worksheet: Worksheet, data: List[Dict], batch_size: int) -> None:
    """Обновляет лист пачками."""
    headers = list(data[0].keys())
    values = [headers] + [list(item.values()) for item in data]
    
    for i in range(0, len(values), batch_size):
        batch = values[i:i + batch_size]
        worksheet.update(f"A{i+1}", batch)

#### 2.4. Расширенная дедупликация

In [ ]:
# 
def merge_and_deduplicate(old: List[Dict], new: List[Dict], key: str) -> List[Dict]:
    """Объединяет данные, сохраняя последнюю версию дубликатов."""
    combined = {item[key]: item for item in old}  # Старые данные
    combined.update({item[key]: item for item in new})  # Перезаписываем новыми
    return list(combined.values())

### 3. Интеграция с многоплатформенным парсингом

#### 3.1. Стратегия для разных сайтов
Подход	       Пример листа	    Метод вызова
Один лист	   All_News	        sheet_name="All_News"
По сайтам	   VC_RU, RBC	    sheet_name="auto"
По категориям  Marketing, Tech	sheet_name=item["category"]

#### 3.2. Пример для VC.ru + RBC

In [ ]:
# Парсим VC.ru
vc_data = parse_articles(vc_html, host="https://vc.ru")
export_to_google_sheets(vc_data, "123abc", sheet_name="auto")

# Парсим RBC
rbc_data = parse_articles(rbc_html, host="https://rbc.ru")
export_to_google_sheets(rbc_data, "123abc", sheet_name="auto")

### 4. Дополнительные улучшения

#### 4.1. Валидация данных

In [ ]:
# 
def validate_data(data: List[Dict]) -> bool:
    """Проверяет обязательные поля."""
    required_fields = {"title", "link", "date"}
    return all(required_fields <= set(item.keys()) for item in data)

#### 4.2. Логирование изменений

In [ ]:
# 
def log_changes(added: int, updated: int, sheet_name: str) -> None:
    """Пишет историю изменений в отдельный лист."""
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "sheet": sheet_name,
        "action": f"Added: {added}, Updated: {updated}"
    }
    # Запись в лист 'Logs'
    ...

#### 4.3. Обновление только новых данных

In [ ]:
# Оптимизация для больших таблиц:
last_update = worksheet.acell("A1").value  # Дата последнего обновления
new_items = [item for item in data if item["date"] > last_update]

### 5. Пример итоговой структуры Google Sheets
Лист	Назначение
VC_RU	Новости с vc.ru
RBC 	Новости с rbc.ru
Logs	История обновлений
Config	Настройки (например, dedup_key)

### 6. Рекомендации
1. Разделение данных:
Используйте отдельные листы, если источники/категории не связаны между собой.
Для общего анализа удобнее один лист с полем source.

2. Безопасность:
Храните credentials.json вне репозитория (добавьте в .gitignore).
Ограничьте права сервисного аккаунта в Google Cloud.

3. Производительность:
Для больших объёмов данных (>10K строк) используйте Google Sheets API batchUpdate.

## 6. START_PARSING
Функция запуска парсинга (основная объединяющая функция)

### 1. Улучшенная start_parsing() с обработкой ошибок

In [ ]:
# 
from typing import Optional, Dict
import traceback
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton

def start_parsing(resume_from: Optional[str] = None) -> Dict[str, int]:
    """
    Основная функция парсинга с возможностью возобновления.
    
    Args:
        resume_from: Точка возобновления ('dynamic_parse', 'parse_articles', etc.)
        
    Returns:
        Словарь с результатами: {
            'total_articles': int,
            'unique_articles': int,
            'spreadsheet_url': str
        }
    """
    # Конфигурация
    config = {
        'chat_id': "123456789",
        'host': "https://vc.ru",
        'spreadsheet_id': "1XQBizX0YVDsZ7PaHlNvnEVLwyOxUZuFJo0447ku6Qdg",
        'max_scrolls': 50
    }
    
    # Результаты
    result = {
        'total_articles': 0,
        'unique_articles': 0,
        'spreadsheet_url': f"https://docs.google.com/spreadsheets/d/{config['spreadsheet_id']}"
    }
    
    try:
        # Этап 1: Стартовое сообщение
        if not resume_from:
            send_telegram_message(
                chat_id=config['chat_id'],
                text=f"🚀 Запуск парсера VC.ru\nВремя: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
                action="start"
            )

        # Этап 2: Динамический парсинг
        if not resume_from or resume_from == 'dynamic_parse':
            html = dynamic_parse(max_scrolls=config['max_scrolls'])
            save_html(html, "data/interim/dynamic_page.html")
            resume_from = None  # Сброс после успешного выполнения

        # Этап 3: Парсинг статей
        if not resume_from or resume_from == 'parse_articles':
            articles_data = parse_articles(html, config['host'])
            result['total_articles'] = len(articles_data)
            resume_from = None

        # Этап 4: Обновление данных
        if not resume_from or resume_from == 'update_data':
            update_data(articles_data)
            resume_from = None

        # Этап 5: Экспорт в Google Sheets
        if not resume_from or resume_from == 'export_to_google_sheets':
            result['unique_articles'] = export_to_google_sheets(
                data_source=articles_data,
                spreadsheet_id=config['spreadsheet_id'],
                sheet_name="VC.ru News"
            )
            
        # Успешное завершение
        send_telegram_message(
            chat_id=config['chat_id'],
            text=f"✅ Парсинг завершен!\nУникальных записей: {result['unique_articles']}",
            action="finish",
            spreadsheet_url=result['spreadsheet_url']
        )
        
        return result

    except Exception as e:
        error_info = {
            'error': str(e),
            'traceback': traceback.format_exc(),
            'resume_point': resume_from or 'dynamic_parse'
        }
        
        # Отправка ошибки в Telegram
        send_telegram_message(
            chat_id=config['chat_id'],
            text=f"🔥 Ошибка на этапе '{resume_from or 'dynamic_parse'}': {e}",
            action="error",
            error_info=error_info
        )
        
        # Сохранение состояния для возобновления
        save_failure_state(resume_from, articles_data if 'articles_data' in locals() else None)
        raise  # Можно убрать для автоматического продолжения при следующем запуске

### 2. Улучшенные Telegram-уведомления

In [ ]:
# 
def send_telegram_message(
    chat_id: str,
    text: str,
    action: str = "info",
    spreadsheet_url: Optional[str] = None,
    error_info: Optional[Dict] = None
) -> None:
    """
    Отправляет интерактивные сообщения в Telegram.
    
    Args:
        action: Тип действия ('start', 'finish', 'error', 'status')
    """
    bot = telebot.TeleBot(config.token)
    markup = None
    
    # Кнопки для разных сценариев
    if action == "finish" and spreadsheet_url:
        markup = InlineKeyboardMarkup()
        markup.add(InlineKeyboardButton("📊 Открыть Google Sheets", url=spreadsheet_url))
        
    elif action == "error":
        markup = InlineKeyboardMarkup()
        markup.add(
            InlineKeyboardButton("🔄 Повторить попытку", callback_data="retry"),
            InlineKeyboardButton("🔍 Подробности", callback_data=f"error_details:{error_info['resume_point']}")
        )
    
    # Отправка сообщения
    try:
        bot.send_message(
            chat_id,
            text,
            reply_markup=markup,
            parse_mode="Markdown"
        )
    except Exception as e:
        logging.error(f"Ошибка отправки в Telegram: {e}")
    finally:
        bot.stop_polling()

# Обработчик кнопок
@bot.callback_query_handler(func=lambda call: True)
def handle_buttons(call):
    if call.data == "retry":
        bot.send_message(call.message.chat.id, "Повторяю парсинг...")
        start_parsing(resume_from=call.message.text.split("'")[-2])
    elif call.data.startswith("error_details"):
        error_point = call.data.split(":")[1]
        bot.send_message(
            call.message.chat.id,
            f"Последняя точка сохранения: `{error_point}`\n"
            f"Для возобновления выполните:\n```python\nstart_parsing(resume_from='{error_point}')\n```",
            parse_mode="Markdown"
        )

### 3. Механизм возобновления работы

In [ ]:
# 
import pickle
from pathlib import Path

def save_failure_state(resume_point: str, data: Optional[List[Dict]] = None) -> None:
    """Сохраняет состояние для возобновления."""
    state = {
        'resume_from': resume_point,
        'data': data,
        'timestamp': datetime.now().isoformat()
    }
    
    Path("data/state").mkdir(exist_ok=True)
    with open("data/state/last_failure.pkl", "wb") as f:
        pickle.dump(state, f)

def load_failure_state() -> Optional[Dict]:
    """Загружает последнее состояние при ошибке."""
    try:
        with open("data/state/last_failure.pkl", "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        return None

In [ ]:
# 


### 4. Дополнительные улучшения

#### 4.1. Проверка состояния при запуске
Добавьте в начало start_parsing():

In [ ]:
# 
# Автоматическое возобновление
if not resume_from:
    last_state = load_failure_state()
    if last_state:
        choice = input(f"Обнаружено незавершённое выполнение (ошибка на этапе '{last_state['resume_from']}'). Продолжить? (y/n): ")
        if choice.lower() == 'y':
            resume_from = last_state['resume_from']
            articles_data = last_state['data']

#### 4.2. Промежуточные статусы

In [ ]:
# Пример функции для отправки статусов:
def send_status_update(chat_id: str, step: str, details: str) -> None:
    """Отправляет промежуточный статус."""
    steps = {
        'dynamic_parse': "🔄 Динамическая загрузка страницы",
        'parse_articles': "📝 Парсинг новостей",
        'update_data': "💾 Сохранение данных",
        'export': "📤 Экспорт в Google Sheets"
    }
    
    send_telegram_message(
        chat_id=chat_id,
        text=f"{steps.get(step, '🚀')} {details}",
        action="status"
    )

### 5. Пример использования

#### 5.1. Ручной запуск

In [ ]:
# 
if __name__ == "__main__":
    start_parsing()  # Обычный запуск
    # Или с возобновлением:
    # start_parsing(resume_from='parse_articles')

#### 5.2. Через планировщик задач

In [ ]:
# Для Windows (run.bat):
# bat
@echo off
call venv\Scripts\activate
python -c "from main import start_parsing; start_parsing()"

### 6. Итоговая архитектура уведомлений
Тип сообщения	Кнопки	               Пример текста
Старт	        -	                   "🚀 Начало парсинга VC.ru в 12:00"
Ошибка	        Повторить/Подробности  "🔥 Ошибка парсинга: timeout"
Статус	        -	                   "📝 Парсинг: 42 новости обработаны"
Завершение	    Открыть Google Sheets  "✅ Успешно! 100 записей"

### Рекомендации
1. Логирование:
Сохраняйте полный лог в файл logs/parsing_<date>.log.

2. Конфигурация:
Вынесите chat_id, spreadsheet_id в .env или config.py.

3. Тестирование:
Добавьте mock-тесты для Telegram API.

### Эти изменения дают:

✅ Возобновление после сбоев
✅ Подробные уведомления в Telegram
✅ Интерактивные элементы управления
✅ Гибкую конфигурацию